In [1]:
# 문자 단위 RNN : 단어를 문자의 연속으로 읽어 들여서 각 단계의 예측과 hidden state를 출력하고 다음 단계에 hidden state 전달
# 단어가 속한 클래스로 출력이 되도록 최종 예측 선택

In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import glob
import os

def findFiles(path): 
    return glob.glob(path)

print(findFiles('../data/02_torch_text_data/names/*.txt'))

import unicodedata
import string

all_letters = string.ascii_letters + ".,;'"
n_letters = len(all_letters)

# 유니코드 문자열을 ASCII 문자열로 변환

def unicodeToascii(s):
    return "".join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters)

print(unicodeToascii('Ślusàrski'))

# 각 언어의 이름 목록인 category_lines 사전 생성

category_lines = {}
all_categories = []

# 파일을 읽고 줄 단위로 분리

def readLines(filename):
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [unicodeToascii(line) for line in lines]

for filename in findFiles('../data/02_torch_text_data/names/*.txt'):
    category = os.path.splitext(os.path.basename(filename))[0]
    all_categories.append(category)
    lines = readLines(filename)
    category_lines[category] = lines
    
n_categories = len(all_categories)

['../data/02_torch_text_data/names/Czech.txt', '../data/02_torch_text_data/names/German.txt', '../data/02_torch_text_data/names/Arabic.txt', '../data/02_torch_text_data/names/Japanese.txt', '../data/02_torch_text_data/names/Chinese.txt', '../data/02_torch_text_data/names/Vietnamese.txt', '../data/02_torch_text_data/names/Russian.txt', '../data/02_torch_text_data/names/French.txt', '../data/02_torch_text_data/names/Irish.txt', '../data/02_torch_text_data/names/English.txt', '../data/02_torch_text_data/names/Spanish.txt', '../data/02_torch_text_data/names/Greek.txt', '../data/02_torch_text_data/names/Italian.txt', '../data/02_torch_text_data/names/Portuguese.txt', '../data/02_torch_text_data/names/Scottish.txt', '../data/02_torch_text_data/names/Dutch.txt', '../data/02_torch_text_data/names/Korean.txt', '../data/02_torch_text_data/names/Polish.txt']
Slusarski


In [2]:
# 각 category(언어)를 line(이름)에 매칭하는 사전인 category_lines 제작 완료
# 나중에 참조가능하도록 all_categories와 n_categories도 추적함

print(category_lines['Italian'][:5])

['Abandonato', 'Abatangelo', 'Abatantuono', 'Abate', 'Abategiovanni']


In [5]:
# 이름을 Tensor로 변경

import torch
import numpy as np
# all_letters로 문자의 주소 찾기 : ex) "a" = 0
def letterToindex(letter):
    return all_letters.find(letter)    # all_letteres의 인덱스로 문자 주소 사용

# 검증을 위해 한 개의 문자를 <1 * n_letters> Tensor로 변환
def letterTotensor(letter):
    tensor = torch.zeros(1,n_letters)
    # tensor = torch.zeros(1, n_letters)
    tensor[0][letterToindex(letter)] = 1
    return tensor

# 한 line(이름)을 <line_length * 1 * n_letters> 혹은 one-hot 문자 벡터 array로 변경

def lineTotensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    # tensor = torch.zeros(len(line), 1, n_letters)
    for li, letter in enumerate(line):
        tensor[li][0][letterToindex(letter)] = 1
    return tensor

print(letterTotensor('J'))

print(lineTotensor('Jones').shape)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0.]]
(5, 1, 56)


In [6]:
# 네트워크 생성
# Autograd 전에, Torch에서 RNN 생성 : 여러 시간 단계에 거쳐 매개변수 복제하는 작업 포함함
# 계층 : hidden state와 Gradient를 가짐

In [ ]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden
    
    def initHidden(self):
        return np.zeros((1, self.hidden_size))
    
n_hidden = 128

rnn = RNN(n_letters, n_hidden, n_categories)

In [ ]:
# 네트워크 한 단계 실행하기 위해
# 입력 (현재 문자 Tensor) + 이전의 hidden state(처음에는 0으로 초기화) 전달 필요
# 출력 (각 언어의 등장 확률) + 다음 hidden state를 반환 받음

input = letterTotensor('A')
hidden = torch.zeros(1, n_hidden)

output, next_hidden = rnn(input, hidden)

In [ ]:
# 효율성을 위해 ~ 매 단계마다 새로운 Tensor를 만들고 싶지 않으므로 
# letterTotensor 대신 lineTotensor를 잘라서 사용
# Tensor의 pre-computing 배치에 의해 더욱 최적화될 수 있음

input = lineTotensor('Albert')
hidden = torch.zeros(1, n_hidden)

output, next_hidden = rnn(input[0], hidden)

In [ ]:
# 학습 준비
# 네트워크의 출력 (각 카테고리의 likelihood)을 통해 가장 확률이 높은 카테고리 이름(언어), 카테고리 번호 반환

def categoryFromoutput(output):
    top_n, top_i = output.topk(1)   # Tensor의 가장 큰 값 및 주소
    category_i = top_i[0].item()    # Tensor에서 정수 값으로 변경
    return all_categories[category_i], category_i

print(categoryFromoutput(output))

In [ ]:
# 학습 예시를 얻는 빠른 방법

import random

def randomChoice()